In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import pickle
with open('/content/drive/My Drive/assignment/Assignment(Taxi_demand)/final_train','rb') as f:
    X_train = pickle.load(f)
f.close()

with open('/content/drive/My Drive/assignment/Assignment(Taxi_demand)/final_test','rb') as f:
    X_test = pickle.load(f)
f.close()

with open('/content/drive/My Drive/assignment/Assignment(Taxi_demand)/train_output','rb') as f:
    y_train = pickle.load(f)
f.close()

with open('/content/drive/My Drive/assignment/Assignment(Taxi_demand)/test_output','rb') as f:
    y_test = pickle.load(f)
f.close()

In [3]:
print(X_train.shape,len(y_train))
print(X_test.shape,len(y_train))

(366760, 18) 366760
(157200, 18) 366760


#**Xgboost Regressor**

In [0]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

In [11]:
params = {'max_depth':[3,4,5,6,7],
          'min_child_weight':[0.01,0.1,1,10],
          'n_estimators':[50,100,200,500]}

model  = xgb.XGBRegressor(booster='gbtree')
grid = RandomizedSearchCV(model, param_distributions=params, scoring= 'neg_mean_squared_error', \
                    n_jobs=-1,cv=5,return_train_score=True) 
                                                
grid.fit(X_train,y_train) 

[12:55:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=None,
                                          objective='reg:linear',
                                          random_state=0, reg_alpha=0,
                                          reg_lambda=1, scale_pos_weight=1,
                                          seed=None, silent=None, subsample=1,
                                  

In [12]:
grid.cv_results_

{'mean_fit_time': array([116.54849806, 124.67089028,  68.34275899,  25.63849435,
        158.6062192 ,  23.66749196,  44.82021403, 100.1795248 ,
         29.93718214,  23.8979085 ]),
 'mean_score_time': array([1.01076555, 0.78304367, 0.4782217 , 0.21109576, 1.32986236,
        0.19110599, 0.41635866, 0.82318964, 0.23510399, 0.19346023]),
 'mean_test_score': array([-167.76173703, -170.18671562, -167.17981207, -167.17909985,
        -169.78063921, -167.17909985, -166.67619979, -167.4611726 ,
        -166.78959842, -167.17909985]),
 'mean_train_score': array([-152.13410523, -132.76289661, -152.63403131, -156.73158851,
        -145.04318115, -156.73158851, -152.6001244 , -143.07684171,
        -152.94755721, -156.73158851]),
 'param_max_depth': masked_array(data=[3, 7, 4, 5, 4, 5, 5, 6, 6, 5],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_min_child_weight': masked_ar

In [13]:
grid.best_params_

{'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 100}

In [14]:
grid.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [15]:
model = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

model.fit(X_train,y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

final_result_train = (mean_absolute_error(y_train,y_train_pred))/(sum(y_train)/len(y_train))
final_result_test = (mean_absolute_error(y_test,y_test_pred))/(sum(y_test)/len(y_test))

print("MAPE for training data : ",final_result_train)
print("MAPE for test data : ",final_result_test)

[12:59:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAPE for training data :  0.12940559952465905
MAPE for test data :  0.1380753575011969
